Make sure you have `CYTools` installed! For instructions, see [here](https://cy.tools/docs/getting-started/).

In [55]:
import cytools
import numpy as np
import pandas as pd

We can begin by reading in the Example data.

In [56]:
df = pd.read_pickle('data.p')

Now we can inspect the contents of the `pandas` DataFrame object

In [57]:
df.columns

Index(['h11', 'points', 'heights', 'kahler', 'W0', 'gs', 'QCD_index',
       'basis_QCD_index', 'log10 ma eV', 'log10 f GeV', 'omega', 'theta',
       'log10 TR GeV', 'TR scenario', 'name', 'extra'],
      dtype='object')

We have the following columns:
    
 - `h11`: the Hodge number $h^{1,1}$ of the Calabi-Yau threefold (also the number of $C_4$ axions)
 - `points`: the points of the polytope $\subset N \cong \mathbb{Z}^4$ used to define the ambient toric variety (i.e., the rays of the toric fan)
 - `heights`: the height vector determining the triangulation of the polytope which gives the specific toric fan of the ambient variety containing the Calabi-Yau threefold as the anticanonical hypersurface (for a review of this construction, see e.g. [here](https://arxiv.org/pdf/2211.03823))
 - `kahler`: the chosen Kähler parameters, expressed in the default divisor basis chosen by `CYTools` (discussed further momentarily)
 - `W0`: the value chosen for the Gukov-Vafa-Witten superpotential $W_0$
 - `gs`: the value chosen for the string coupling $g_s$
 - `QCD_index`: the (0-)index of the prime toric divisor chosen to host QCD, in the ordering set by `CYTools`
 - `basis_QCD_index`: the (0-)index of the axion associated to the QCD prime toric divisor, when sorted from largest to smallest instanton scale (smallest to largest instanton action)
 - `log10 ma eV`: the masses of the $h^{1,1}$ axions, expressed as $\log_{10}(m_a/\mathrm{eV})$
 - `log10 f GeV`: the decay constants of the $h^{1,1}$ axions, expressed as $\log_{10}(f_a/\mathrm{GeV})$
 - `omega`: the relic abundance of the $h^{1,1}$ axions in whatever reheating scenario is associated to that row
 - `theta`: the chosen initial misalignment angles for the $h^{1,1}$ axions according to the relevant reheating scenario
 - `log10 TR GeV`: the reheating temperature $T_R$ for the given model according to the relevant reheating scenario (if the scenario is "prompt reheating" and there are no axions heavier than the QCD axion, as is the case for most examples, the reheating temperature is unconstrained)
 - `TR scenario`: the reheating scenario, either "prompt reheating" ($w_R = -1$) or "modulus domination" ($w_R = 0$), see Sec. 2.2 of our paper
 - `example`: the name of the subsubsection header associated with the given example
 - `extra`: any miscellaneous extra data, such as the axion-photon couplings for the birefringence example

Let's look at an example: in particular, let's look at the prompt reheating scenario as discussed in the cosmology illustration section.

In [63]:
datum = df.iloc[1]

In [65]:
datum['name'], datum['TR scenario']

('cosmology illustration', 'prompt reheating')

Let's construct the relevant Calabi-Yau object

In [69]:
polytope = cytools.Polytope(datum['points'])
triangulation = polytope.triangulate(heights=datum['heights'])
cy = triangulation.cy()

We can quickly verify that each of these objects has the right datatype

In [70]:
polytope

A 4-dimensional reflexive lattice polytope in ZZ^4 with points [(0, 0, 0, 0), (0, 1, 0, 0), (-1, 0, 1, -1), (0, 0, 0, 1), (0, 0, 1, 0), (2, -1, -1, 0), (-1, -1, -1, 0), (1, 0, 0, 0), (-1, 0, 0, 0), (0, -1, -1, 0), (1, -1, -1, 0)] which are labelled [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [71]:
triangulation

A fine, regular, star triangulation of a 4-dimensional point configuration with 11 points in ZZ^4

In [72]:
cy

A Calabi-Yau 3-fold hypersurface with h11=6 and h21=46 in a 4-dimensional toric variety

For example, we can print out some parameters

In [107]:
print(f'log10(ma/eV) = {np.round(datum["log10 ma eV"],1).tolist()}')
print(f'log10(f/GeV) = {np.round(datum["log10 f GeV"],1).tolist()}')

log10(ma/eV) = [13.4, -1.3, -8.8, -15.9, -18.0, -20.8]
log10(f/GeV) = [15.6, 15.6, 15.6, 15.3, 15.7, 15.9]


In [108]:
datum["basis_QCD_index"]

2

Because `basis_QCD_index` is 2, we know that the QCD index appears third (!!) in the above list, having a mass of $10^{-8.8}$ eV.

We can also verify some of the geometric numbers reported in Section 4.2.2. For example, we can check the volume of the QCD divisor.

In [109]:
tau = cy.compute_divisor_volumes(datum['kahler'])

In [110]:
print(f'The divisor volumes are {np.round(tau,1).tolist()}; in particular, the QCD divisor volume is {tau[datum["QCD_index"]]:.1f}')

The divisor volumes are [129.9, 56.6, 56.6, 73.3, 34.7, 33.4, 8.4, 19.3, 30.2, 31.5]; in particular, the QCD divisor volume is 34.7


Let's now look at the KK scale in this example.

In [118]:
mPl_GeV = 2.4e18 # Planck mass, in eV
cy_vol = cy.compute_cy_volume(datum['kahler'])
print(f'The KK scale for this model is {mPl_GeV / cy_vol**(2/3):.1e} GeV')

The KK scale for this model is 5.3e+16 GeV


Finally, let's compute the Hubble scale $H_R$ at reheating from $W_0$, using Eq. 3.30 from the paper (letting the Planck mass set the scale of the modulus).

In [123]:
H_R = (datum['gs']**6 * datum['W0']**3 * (1e9 * mPl_GeV)) / (2**(33/2) * np.pi * cy_vol**3)

In [129]:
print(f'3H_R = {3 * H_R:.3f} eV')

3H_R = 0.046 eV


As we discuss in the text, notice how this is comparable to the mass of the lightest heavy axion (i.e., the lightest axion heavier than the QCD axion): this means that if $W_0$ set the reheating scale for the prompt reheating scenario, all axions heavier than the QCD axion would have their abundance exponentially diluted by inflation.

In [133]:
print(f'The mass of the lightest heavy axion is {10.**datum["log10 ma eV"][datum["basis_QCD_index"]-1]:.3f} eV')

The mass of the lightest heavy axion is 0.046 eV
